In [1]:
import pandas as pd
import time
from bs4 import BeautifulSoup as bs
import requests
from datetime import date

In [2]:
site = 'https://www.moex.com'
cap_page = 'http://www.moex.com/s26'
f_patch = 'D:/YandexDisk/!ИПЭИ/База данных по акциям/MOEX/market_cap.csv'
current_date = date.today()

In [3]:
# Функция получает таблицы капитализации, квартал, год и заголовки Дата Фрейма
# Возвращат Дата Фрейм капитализации за год/квартал
def get_quarter(tr_tabl, q, y, col_name):
    q_cap = pd.DataFrame(columns=col_name)
    for tr in tr_tabl:
        tds = tr.find_all('td')
        if len(tds) == 0:
            continue
        else:
            pp = 1
            row = []
            for td in tds:
                if pp == 5 or pp == 6 or pp == 7:
                    s = (td.text).replace(u'\xa0', u' ')
                    s = s.replace(" ", "")
                    s = s.replace(",", ".")
                    if len(s) != 0:
                        s = float(s)
                else:
                    s = td.text
                row.append(s)
                pp = pp + 1
        row.append(q)
        row.append(y)
        ind = len(q_cap)
        q_cap.loc[ind] = row
    return q_cap

In [4]:
# Считываем старый файл
old_data = pd.read_csv(f_patch, sep=';', encoding='cp1251')

In [5]:
# Получаем последний год и квартал
last_year = old_data['year'].unique().max()
last_quarters = old_data[old_data['year'] == last_year]
last_quarter = last_quarters['quarter'].max()

In [9]:
last_quarter

3

In [10]:
# Получаем страницу с капитализацией без Selenium
url = cap_page
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}
response = requests.get(url.strip(), headers=headers, timeout=10)

In [11]:
soup = bs(response.text)
table_all_cap = soup.find('table', {'class': 'table1'})

In [12]:
ths = table_all_cap.find_all('th')

In [13]:
years = []
for th in ths:
    years.append(th.text)

In [14]:
years

['2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 '2022',
 '2023']

In [15]:
trs = table_all_cap.find_all('tr')
trs.pop(0)

<tr>
<th style="text-align:center; white-space:nowrap"><strong>2011</strong></th>
<th style="text-align:center; white-space:nowrap"><strong>2012</strong></th>
<th style="text-align:center; white-space:nowrap"><strong>2013</strong></th>
<th style="text-align:center; white-space:nowrap"><strong>2014</strong></th>
<th style="text-align:center; white-space:nowrap"><strong>2015</strong></th>
<th style="text-align:center; white-space:nowrap"><strong>2016</strong></th>
<th style="text-align:center; white-space:nowrap"><strong>2017</strong></th>
<th style="text-align:center; white-space:nowrap"><strong>2018</strong></th>
<th style="text-align:center; white-space:nowrap"><strong>2019</strong></th>
<th style="text-align:center; white-space:nowrap"><strong>2020</strong></th>
<th><strong>2021</strong></th>
<th>2022</th>
<th>2023</th>
</tr>

In [16]:
# Формируем словарь из переменных год, квартал, ссылка
data = []
for tr in trs:  # для каждой строки
    year = 2011
    tds = tr.find_all('td')  # берем все
    for td in tds:
        dd = []
        dd.append(year)
        if td.text == '\xa0':
            dd.append('nq')  # заполняем пустые кварталы
            dd.append('nl')  # заполняем пустые ссылки
            data.append(dd)
            year = year+1
        else:
            dd.append(td.text)  # значение квартала
            dd.append(td.find('a')['href'])  # ссылка
            data.append(dd)
            year = year+1

# Удаляем слово квартал
for d in data:
    d[1] = d[1][0:1]

# Отщищаем от пустых кварталов
clear_data = []
for d in data:
    if d[1] == 'n':
        continue
    else:
        clear_data.append(d)

# Проверка наличия новых данных
new_data = False
for dt in clear_data:
    if int(dt[0]) == int(last_year):
        if int(dt[1]) > int(last_quarter):
            print('Новые данные')
            new_data = True
            break
        else:
            continue
    if int(dt[0]) > int(last_year):
        print('Новые данные')
        new_data = True
        break
if new_data != True:
    print('Новых данных нет')
else:  # делаем сохранение старых данных
    old_data.to_csv(
        f'D:/YandexDisk/!ИПЭИ/База данных по акциям/MOEX/backup/market_cap_{current_date}.csv', sep=';', encoding='cp1251', index=False)

Новые данные


In [17]:
# Формируем пустой Дата Фрейм
col = ['ticker', 'name', 'cat', 'reg',
       'volume', 'price', 'cap', 'quarter', 'year']
if len(old_data) == 0:
    cap = pd.DataFrame(columns=col)
else:
    cap = old_data

In [21]:
# Получаем данные о капитализации
skip = []  # будем записывать уникальные ссылки
for q_data in clear_data:
    # По условиям проверяем если, данные за год/квартал в считанном файле
    if int(q_data[0]) <= int(last_year) and int(q_data[1]) <= int(last_quarter):
        continue
    if int(q_data[0]) == int(last_year) and int(q_data[1]) <= int(last_quarter):
        continue
    # if int(q_data[0]) < int(last_year):
    #    continue
    # if int(q_data[1]) <= int(last_quarter) and int(q_data[0]) == int(last_year):
    #    continue
    # Проверяем подходит ли ссылка для формирования запроса
    if len(q_data[2]) < 7:
        url = site + q_data[2]
        response = requests.get(url.strip(), headers=headers, timeout=10)
        soup = bs(response.text)
        q_table = soup.find('table', {'class': 'table1'})
        tr_table = q_table.find_all('tr')
        data_q = get_quarter(tr_table, q_data[1], q_data[0], col)
        cap = pd.concat([cap, data_q], ignore_index=True)
        time.sleep(5)
    else:
        # Если ссылка не подошла, добавляем ее в пропущенные
        skip.append(q_data[2])

In [22]:
if len(skip) != 0:
    for link in skip:
        print('Пропущенна ссылка', link)
else:
    print('Пропущенных ссылок нет')

Пропущенна ссылка https://fs.moex.com/f/17268/marketcap2022-3q.xlsx
Пропущенна ссылка http://moex.com/a8501


In [23]:
clear_data

[[2011, '4', '/a20'],
 [2012, '4', '/a1465'],
 [2013, '4', '/a2261'],
 [2014, '4', '/a2987'],
 [2015, '4', '/a3503'],
 [2016, '4', '/a3882'],
 [2017, '4', '/a4258'],
 [2018, '4', '/a6401'],
 [2019, '4', '/a7150'],
 [2020, '4', '/a7605'],
 [2021, '4', '/a8067'],
 [2022, '4', '/a8313'],
 [2012, '3', '/a1244'],
 [2013, '3', '/a2045'],
 [2014, '3', '/a2813'],
 [2015, '3', '/a3369'],
 [2016, '3', '/a3794'],
 [2017, '3', '/a4184'],
 [2018, '3', '/a4420'],
 [2019, '3', '/a7080'],
 [2020, '3', '/a7396'],
 [2021, '3', '/a7935'],
 [2022, '3', 'https://fs.moex.com/f/17268/marketcap2022-3q.xlsx'],
 [2023, '3', '/a8593'],
 [2012, '2', '/a1127'],
 [2013, '2', '/a1823'],
 [2014, '2', '/a2702'],
 [2015, '2', '/a3177'],
 [2016, '2', '/a3691'],
 [2017, '2', '/a4094'],
 [2018, '2', '/a4377'],
 [2019, '2', '/a6997'],
 [2020, '2', '/a7305'],
 [2021, '2', '/a7798'],
 [2022, '2', '/a8177'],
 [2023, '2', 'http://moex.com/a8501'],
 [2012, '1', '/a685'],
 [2013, '1', '/a1688'],
 [2014, '1', '/a2474'],
 [2015, '

In [24]:

#Добавление в Дата Фрейм данные из xlsx файла
data2022_3 = pd.read_excel('https://fs.moex.com/f/17268/marketcap2022-3q.xlsx', header=2)
data2022_3_new = data2022_3.dropna(thresh=3).copy()
data2022_3_new['quarter']='3'
data2022_3_new['year']='2022'
col_table = []
for colum in data2022_3_new.columns:
    col_table.append(colum)
col_new = dict(zip(col_table, col))
data2022_3_new = data2022_3_new.rename(columns = col_new)
cap = pd.concat([cap,data2022_3_new], ignore_index=True)


In [25]:

#Добавление пропущеенной ссылки
url = 'http://moex.com/a8501'
response = requests.get(url.strip(), headers=headers, timeout=10)
soup = bs(response.text)
q_table = soup.find('table', {'class':'table1'})
tr_table = q_table.find_all('tr')
data_q = get_quarter (tr_table,2,2023,col)
cap = pd.concat([cap,data_q], ignore_index=True)


In [26]:
# Запись нового файла
if new_data == True:
    cap.to_csv('td.csv', sep=';', encoding='cp1251', index=False)